Idea of cutset conditioning: it's a way to run exact inference on a model with loops. You cut the loop by observing one of the variables in the loop to all the possible states, then fuse the results in a smart way.

 Cutset Conditioning is a technique for solving nearly-tree-structured CSPs in which some variables are assigned to separately from the rest, removed from the constraint graph, and leaving a tree-structured CSP for those remaining.

 Cutsets are some set of variables that are cut (severing edges) from the original constraint graph and solved separately.

 Conditioning is the process of assigning a value to some variable in a cutset, performing forward checking on its neighbor domains before cutting, and finally, severing it from the original graph.

https://forns.lmu.build/classes/spring-2019/cmsi-282/lecture-13M.html#backtracking++

In [1]:
import src.data.breathe_data as bd

# import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.data.helpers as dh

# import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots
import plotly.express as px

# import src.models.helpers as mh

# import pandas as pd
import numpy as np

Figure per entry that has the AR from obs FEF2575 on top and on the bottom the point mass AR obtained by repeating model runs with several point mass HFEV1 (3, 3.5, 4, 4.5, 5, etc)

In [2]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx")

INFO:root:* Checking for same day measurements *


In [77]:
# With each run I should retrieve
# 1/ the message from FEF25-75%FEFV1 to AR
# 2/ the point mass message from the factor ecFEV1, HFEV1 to AR
# Use the point in time model, there is no shared variables.


def can_messages_align_for_ID(df_for_ID):
    df_for_ID.reset_index(inplace=True, drop=True)
    height = df_for_ID.loc[0, "Height"]
    age = df_for_ID.loc[0, "Age"]
    sex = df_for_ID.loc[0, "Sex"]
    id = df_for_ID.loc[0, "ID"]
    (
        model,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(height, age, sex)

    FEV_to_AR_key = "['ecFEV1 (L)', 'Healthy FEV1 (L)', 'Airway resistance (%)'] -> Airway resistance (%)"
    FEF2575_to_AR_key = (
        "['ecFEF25-75 % ecFEV1 (%)', 'Airway resistance (%)'] -> Airway resistance (%)"
    )

    HFEV1_obs_list = [2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5]
    colour_list = px.colors.sample_colorscale(
        "YlGnBu", [i / (len(HFEV1_obs_list) - 1) for i in range(len(HFEV1_obs_list))]
    )

    df_for_ID = df_for_ID.sort_values(by="ecFEF2575%ecFEV1", ascending=True)
    # Take 4 idx in 5, 30, 60, 95 percentiles of the data
    idx_list = list((len(df_for_ID) * np.array([0.05, 0.5, 0.95])).astype(int))
    df_for_ID_sub = df_for_ID.iloc[idx_list, :]

    res_per_idx = []

    for idx in df_for_ID_sub.index:
        FEV1_obs = df_for_ID.loc[idx, "ecFEV1"]
        FEF2575prctFEV1_obs = df_for_ID.loc[idx, "ecFEF2575%ecFEV1"]
        FEV_m_list = []

        # Query AR
        for HFEV1_obs in HFEV1_obs_list:
            # HFEV1_obs must be > ecFEV1_obs
            evidence = [
                [ecFEV1, FEV1_obs],
                [ecFEF2575prctecFEV1, FEF2575prctFEV1_obs],
                [HFEV1, HFEV1_obs],
            ]
            _, messages = ih.infer_on_factor_graph(
                inf_alg, [AR], evidence, get_messages=True
            )

            FEV_m_list.append(messages[FEV_to_AR_key])
            FEF2575_m = messages[FEF2575_to_AR_key]

        res_per_idx.append([FEV1_obs, FEF2575prctFEV1_obs, FEV_m_list, FEF2575_m])

    fig = make_subplots(rows=6, cols=1, vertical_spacing=0.05)
    plot_row = 1
    for FEV1_obs, FEF2575prctFEV1_obs, FEV_m_list, FEF2575_m in res_per_idx:

        for HFEV1_obs, FEV_m, colour in zip(HFEV1_obs_list, FEV_m_list, colour_list):
            ih.plot_histogram(
                fig,
                AR,
                FEV_m,
                AR.a,
                AR.b,
                plot_row,
                1,
                name=f"HFEV1 = {HFEV1_obs}",
                annot=False,
            )
            # Change the last trace's colour
            fig.data[-1].marker.color = colour
            # Hide legend if plot_row > 1
            if plot_row > 1:
                fig.data[-1].showlegend = False

        ih.plot_histogram(
            fig,
            AR,
            FEF2575_m,
            AR.a,
            AR.b,
            plot_row + 1,
            1,
            annot=False,
            title=AR.name,
            colour="grey",
        )
        # hide this last trace's legend
        fig.data[-1].showlegend = False
        # Add message from ecFEV1/HFEV1 factor on y axis row 1 title
        fig.update_yaxes(title_text=f"ecFEV1<br>{FEV1_obs:.2f}L", row=plot_row, col=1)
        fig.update_yaxes(title_text=f"ecFEF25-75%ecFEV1<br>{FEF2575prctFEV1_obs:.2f}%", row=plot_row + 1, col=1)
        plot_row += 2

    # Reduce font size and margins
    title = f"ID {id} - Can points mass messages from HFEV1, ecFEV1 align with messages from FEF25-75"
    # Reduce margins between plots
    fig.update_layout(
        font=dict(size=8),
        margin=dict(l=10, r=10, t=30, b=10),
        height=750,
        width=600,
        barmode="overlay",
        bargap=0.1,
        title=title,
    )
    fig.update_xaxes(title_standoff=6)

    fig.write_image(
        dh.get_path_to_main() + f"/PlotsBreathe/Cutset_conditioning/{title}.pdf"
    )
    # fig.show()


interesting_ids = [
    "132",
    "146",
    "177",
    "180",
    "202",
    "527",
    "117",
    "131",
    "134",
    "191",
    "139",
    "253",
    "101",
    # Also from consec values
    "405", 
    "272",
    "201",
    "203"
]

df[df.ID.isin(interesting_ids)].groupby("ID").apply(can_messages_align_for_ID)

# df_for_ID = df[df.ID == "101"]
# can_messages_align_for_ID(df_for_ID)

/Users/tristan.trebaol/Desktop/PhD/Code/pgmpy/pgmpy/inference/ExactInference.py:1562: RuntimeWarning:

invalid value encountered in divide



""
